In [ ]:
#訓練集用來模型訓練，驗證集用來調整參數，而測試集用來衡量模型表現好壞。

X = train_clean.drop(columns=['CUS_ID','Y1'])
y = train_clean['Y1']

X_trainval,X_test,y_trainval,y_test = train_test_split(X,y,test_size = 0.25 , random_state=0) 
X_train,X_val,y_train,y_val = train_test_split(X_trainval,y_trainval,test_size = 0.25,random_state=1) 
print("Size of training set:{} size of validation set:{} size of teseting set:{}".format(X_train.shape[0],X_val.shape[0],X_test.shape[0]))



In [ ]:
# GridSearch 自動調參
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

X, y = train, train_y

cv_params = {'n_estimators': [400, 500, 600, 700, 800]}
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.25, random_state=33)

xgbc = XGBClassifier(eta = 0.15, gamma =0.2,  min_child_weight = 10 , max_depth =4 , subsample = 0.7 ,
                    n_estimators = 500, colsample_bytree = 0.7 , alpha = 0.8 , lambda_reg = 0.8, scoring = 'roc_auc', 
                     learning_rate = 0.1 , objective= 'binary:logistic')

#xgbc.fit(X_train , y_train)

#ans = list(xgbc.predict(test))

optimized_GBM = GridSearchCV(xgbc, param_grid=cv_params, scoring='roc_auc', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.grid_scores_
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

看起來切三份使得Y1=1的樣本數更小了，因此不管參數怎麼調整，模型依然會全部預測0  
在此情況下，應該做  
1.改善train data imbalance的情況(oversampling)  
2.改善餵給model的資料(多餵一點Y1=1)  
3.改善填值的方式(ex:MICE、Median)  
4.改善Model的選擇(可以試試看logistic、randomforest、Naive Bayes)  

In [ ]:
%%time
XGBC_3 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.1, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

XGBC_3.fit(X_trainval,y_trainval) #使用訓練集和驗證集進行訓練，more data always results in good performance. 

print('best_score:',roc_auc_score(y_test,XGBC_3.predict(X_test)))
